In [134]:
from transformers import AutoTokenizer, AutoModel

# we are going to fine tune :  google-bert/bert-base-cased
checkpoint = "google-bert/bert-base-cased"


In [135]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [136]:
ins = tokenizer("I love you", padding=True, truncation=True, return_tensors="pt")

### LOAD YELP REVIEW DATASET

In [137]:
from datasets import load_dataset

dataset = load_dataset("Yelp/yelp_review_full")

In [138]:
dataset["train"][39]

{'label': 3,
 'text': "Great breakfast, good price. You might have to stand outside in line though, so I don't really recommend winter time to go. lol. Very friendly service, interesting coffee mugs. They have great deserts and such also. Bring your cash though as they dont' take cards."}

In [139]:
# convert the raw dataset into tensor

def tokenize(example):
    return tokenizer(example["text"], padding=True, truncation=True)


tokenized_dataset = dataset.map(tokenize, batched=True)

In [140]:
tokenized_dataset = tokenized_dataset.remove_columns(["text"])

In [141]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
})

In [142]:
# rename column label to labels
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")

In [143]:
tokenized_dataset["train"][0]

{'labels': 4,
 'input_ids': [101,
  173,
  1197,
  119,
  2284,
  2953,
  3272,
  1917,
  178,
  1440,
  1111,
  1107,
  170,
  1704,
  22351,
  119,
  1119,
  112,
  188,
  3505,
  1105,
  3123,
  1106,
  2037,
  1106,
  1443,
  1217,
  10063,
  4404,
  132,
  1119,
  112,
  188,
  1579,
  1113,
  1159,
  1107,
  3195,
  1117,
  4420,
  132,
  1119,
  112,
  188,
  6559,
  1114,
  170,
  1499,
  118,
  23555,
  2704,
  113,
  183,
  9379,
  114,
  1134,
  1139,
  2153,
  1138,
  3716,
  1106,
  1143,
  1110,
  1304,
  1696,
  1107,
  1692,
  1380,
  5940,
  1105,
  1128,
  1444,
  6059,
  132,
  1105,
  1128,
  1169,
  1243,
  5991,
  16179,
  1106,
  1267,
  18137,
  1443,
  1515,
  1106,
  1267,
  1140,
  1148,
  119,
  1541,
  117,
  1184,
  1167,
  1202,
  1128,
  1444,
  136,
  178,
  112,
  182,
  2807,
  1303,
  1774,
  1106,
  1341,
  1104,
  1251,
  11344,
  178,
  1138,
  1164,
  1140,
  117,
  1133,
  178,
  112,
  182,
  1541,
  4619,
  170,
  9153,
  119,
  102,
  0,
  0,

In [144]:
tokenized_dataset.set_format("torch")

In [145]:
# create dataloader to train in batchs
from torch.utils.data import DataLoader

tokenized_dataloader =  DataLoader(tokenized_dataset["train"], batch_size=8, shuffle=True)
eval_dataloader = DataLoader(tokenized_dataset["test"], batch_size=8)

In [146]:
len(tokenized_dataset["train"])

650000

In [147]:
len(tokenized_dataloader)

81250

In [148]:
# Load model for sequnece classification
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [149]:
# create optimizer and lr
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)


In [150]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(tokenized_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [151]:
lr_scheduler

In [152]:
import torch
from accelerate.test_utils.testing import get_backend

device = "cpu"

model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [153]:
next(iter(tokenized_dataloader))

{'labels': tensor([0, 3, 1, 4, 0, 0, 3, 3]),
 'input_ids': tensor([[  101, 14856, 20737,  ...,   119,   119,   102],
         [  101,  6467,   170,  ...,     0,     0,     0],
         [  101, 23878,  1303,  ...,     0,     0,     0],
         ...,
         [  101,  1247,  1110,  ...,     0,     0,     0],
         [  101,  1135,  1108,  ...,     0,     0,     0],
         [  101,   146,   112,  ...,     0,     0,     0]]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]])}

In [154]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in tokenized_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 14/243750 [3:19:41<57942:27:04, 855.81s/it]


KeyboardInterrupt: 

In [110]:
device

'mps'

In [132]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e